In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from pathlib import Path
import tmdbsimple as tmdb
import json

project_path = Path().cwd().parent.parent
data_path = project_path / "data"

In [2]:
tmdb.API_KEY = 'f242e2c5e2deab7ec67acb9a3722f983'

movies = pd.read_csv(data_path / "tmdb-movie-metadata" / "tmdb_5000_movies.csv")
credits = pd.read_csv(data_path / "tmdb-movie-metadata" / "tmdb_5000_credits.csv")

In [4]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [149]:
# cleaning of movies dataset
df = movies.copy()
df.keywords = df.keywords.map(lambda keywords: ' '.join([keyword['name'] for keyword in json.loads(keywords)]))

In [186]:
# Create production_countries dataset
production_countries = flatten([json.loads(item) for index, item in df.production_countries.iteritems()])
production_countries = pd.DataFrame(production_countries).drop_duplicates()\
                                                         .reset_index(drop=True)
production_countries.to_csv(data_path / 'production_countries.csv')

In [215]:
# Create countries_produce_movies dataset
produce = pd.DataFrame({'production_country_id':[], 'movie_id':[]})
for index, row in movies.iterrows():
    countries = json.loads(row['production_countries'])
    for country in countries:
        code = country['iso_3166_1']
        production_country_id = production_countries[production_countries.iso_3166_1 == code].index.item()
        produce = produce.append({'production_country_id': int(production_country_id), 'movie_id': int(row['id'])}, ignore_index=True)

produce.production_country_id = produce.production_country_id.astype(np.int64)
produce.movie_id = produce.movie_id.astype(np.int64)
produce.to_csv(data_path / 'countries_produce_movies.csv', index=False)

In [220]:
# Create production_companies dataset
production_companies = flatten([json.loads(item) for index, item in movies.production_companies.iteritems()])
production_companies = pd.DataFrame(production_companies).drop_duplicates()\
                                                         .set_index('id')\
                                                         .sort_index()

production_companies.to_csv(data_path / 'production_companies.csv')

In [227]:
# Create companies_produce_movies dataset
produce = pd.DataFrame({'production_company_id':[], 'movie_id':[]})
for index, row in movies.iterrows():
    companies = json.loads(row['production_companies'])
    for companie in companies:
        produce = produce.append({'production_company_id': companie['id'], 'movie_id': row['id']}, ignore_index=True)

produce.production_company_id = produce.production_company_id.astype(np.int64)
produce.movie_id = produce.movie_id.astype(np.int64)
produce.to_csv(data_path / 'companies_produce_movies.csv', index=False)

In [5]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [244]:
actors.shape

(2929, 13)